In [ ]:
import os
import json
import random

In [ ]:
# 무신사 카테고리
CLOTHING_CATEGORIES = {
    "cold": {
        "outer": [
            ("002012", "숏패딩/숏헤비 아우터"),
            ("002024", "겨울 더블 코트"),
            ("002007", "겨울 싱글 코트"),
            ("002013", "롱패딩/롱헤비 아우터"),
            ("002023", "플리스/뽀글이"),
            ("002025", "무스탕/퍼")
        ],
        "top": [
            ("001006", "니트/스웨터"),
            ("001004", "후드 티셔츠")
        ],
        "bottom": [
            ("003002", "데님 팬츠"),
            ("003004", "트레이닝/조거 팬츠"),
            ("003005", "레깅스")
        ]
    },
    "cool": {
        "outer": [
            ("002023", "플리스/뽀글이"),
            ("002007", "겨울 싱글 코트"),
            ("002001", "블루종/MA-1"),
            ("002003", "슈트/블레이저 재킷"),
            ("002002", "레더/라이더스 재킷"),
            ("002017", "트러커 재킷")
        ],
        "top": [
            ("001005", "맨투맨/스웨트셔츠"),
            ("001002", "셔츠/블라우스"),
            ("001006", "니트/스웨터")
        ],
        "bottom": [
            ("003002", "데님 팬츠"),
            ("003007", "코튼 팬츠")
        ]
    },
    "mild": {
        "outer": [
            ("002017", "트러커 재킷"),
            ("002002", "레더/라이더스 재킷"),
            ("002003", "슈트/블레이저 재킷"),
            ("002019", "아노락 재킷"),
            ("002006", "나일론/코치  재킷")
        ],
        "top": [
            ("001005", "맨투맨/스웨트셔츠"),
            ("001002", "셔츠/블라우스"),
            ("001003", "피케/카라 티셔츠"),
            ("001010", "긴소매 티셔츠")
        ],
        "bottom": [
            ("003007", "코튼 팬츠"),
            ("003010", "점프 슈트/오버올")
        ]
    },
    "hot": {
        "outer": [],
        "top": [
            ("001001", "반소매 티셔츠"),
            ("001011", "민소매 티셔츠"),
            ("001003", "피케/카라 티셔츠")
        ],
        "bottom": [
            ("003009", "숏 팬츠"),
            ("003005", "레깅스")
        ]
    }
}

In [ ]:
# 계절
SEASONAL_COLOR_COMBINATIONS = {
    "봄": {
        "그레이 베이스": ["그레이 톤온톤", "그레이 + 블루", "그레이 + 핑크"],
        "브라운 베이스": ["브라운 톤온톤", "브라운 + 민트", "브라운 + 블루"],
        "블루 베이스": ["블루 톤온톤", "블루 + 화이트", "블루 + 비비드 컬러"],
        "베이지 베이스": ["베이지 + 크림", "베이지 + 레몬", "베이지 + 핑크"],
        "네이비 베이스": ["네이비 + 비비드 컬러", "네이비 + 크림", "네이비 + 민트"],
        "올리브 베이스": ["올리브 + 크림", "올리브 + 브라운", "올리브 + 민트"]
    },
    "여름": {
        "브라운 베이스": ["브라운 + 화이트", "브라운 톤온톤", "브라운 + 블루"],
        "블루 베이스": ["블루 + 화이트", "블루 톤온톤", "블루 + 베이지"],
        "그레이 베이스": ["그레이 + 화이트", "그레이 톤온톤", "그레이 + 테라코타"],
        "올리브 베이스": ["올리브 + 화이트", "올리브 + 브라운", "올리브 톤온톤"],
        "파스텔 베이스": ["파스텔 블루 + 화이트", "파스텔 핑크 + 브라운"]
    },
    "가을,겨울": {
        "그레이 베이스": ["그레이 + 네이비", "그레이 톤온톤", "그레이 + 흑청"],
        "베이지 베이스": ["베이지 + 크림", "베이지 + 브라운", "베이지 + 네이비"],
        "네이비 베이스": ["네이비 톤온톤", "네이비 + 생지 데님", "네이비 + 올리브"],
        "화이트 베이스": ["화이트 + 크림", "화이트 + 라벤더"],
        "올리브 베이스": ["올리브 + 크림", "올리브 + 브라운", "올리브 톤온톤"]
    }
}

In [ ]:
PREVIOUS_CLOTHES_FILE = "previous_clothes.txt"

In [ ]:
def load_previous_clothes():
  """
  어제 입은 옷 불러오는 함수
  """
  if os.path.exists(PREVIOUS_CLOTHES_FILE):
      with open(PREVIOUS_CLOTHES_FILE, "r") as file:
          clothes = file.read().strip().split(",")
          return clothes if clothes != [""] else []  # 빈 파일 처리
  return []

def save_previous_clothes(clothes):
  """
  어제 입었던 옷을 저장하는 함수
  """
  with open(PREVIOUS_CLOTHES_FILE, "w") as file:
      file.write(",".join(clothes))

In [ ]:
def get_all_clothes():
  """
  모든 옷 아이템 리스트 생성하는 함수
  """
  all_clothes = []
  for weather in CLOTHING_CATEGORIES.values():
      for category_items in weather.values():
          all_clothes.extend(item[1] for item in category_items)
  return all_clothes

In [ ]:
import requests
import json

def fetch_weather(city, api_key, lang='kr', units='metric'):
    """
    동기적으로 날씨 정보를 가져오는 함수
    """
    api_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&APPID={api_key}&lang={lang}&units={units}"
    response = requests.get(api_url)
    response.raise_for_status()
    return response.text

def forecast(city):
    """
    동기적으로 날씨 예보를 반환하는 함수
    """
    api_key = "6f99cab6ab003735f909191f543f99b4" #api 키
    response_text = fetch_weather(city, api_key)
    result = json.loads(response_text)

    sky = result['weather'][0]['description']
    temp = result['main']['temp']
    return sky, temp

In [ ]:
def convert_temperature(temp_celsius):
  """
  온도를 섭씨에서 화씨로 변환하는 함수
  """
  if temp_celsius < 0:
      return f"영하 -{int(abs(temp_celsius))}(C)"
  else:
      return f"{int(temp_celsius)}(C)"

In [ ]:
def get_clothing_by_weather(temp):
  """
  날씨에 맞는 옷을 추천하는 함수
  """
  if temp < 5:
      return CLOTHING_CATEGORIES["cold"]
  elif 5 <= temp < 15:
      return CLOTHING_CATEGORIES["cool"]
  elif 15 <= temp < 25:
      return CLOTHING_CATEGORIES["mild"]
  else:
      return CLOTHING_CATEGORIES["hot"] # 날씨는 임의로 설정

In [ ]:
def print_random_recommendation(recommendations, previous_clothes):
    """
    랜덤으로 옷을 추천하는 함수 (전날 입었던 옷만 제외)
    """
    base_url = "https://www.musinsa.com/category/{}" # 무신사 사이트를 통해 추천 아이템의 상품을 추천

    # 이전에 입었던 옷을 제외한 추천 목록 필터링 (전날 입었던 옷만 제외)
    filtered_recommendations = {
        category: [item for item in items if item[1] not in previous_clothes]
        for category, items in recommendations.items()
    }

    # 각 카테고리에서 하나씩 무작위로 선택
    selected_items = {}
    for category in ["outer", "top", "bottom"]:
        if category in filtered_recommendations and filtered_recommendations[category]:
            selected_items[category] = random.choice(filtered_recommendations[category])
        else:
            selected_items[category] = None  # 해당 카테고리에 추천 가능한 아이템이 없을 경우

    # 추천 결과 출력
    print("\n오늘의 추천 옷:")
    for category, item in selected_items.items():
        if item:
            url = base_url.format(item[0])
            print(f"- {category.capitalize()}: [{item[0]}] {item[1]} - {url}")
        else:
            print(f"- {category.capitalize()}: 추천 가능한 아이템이 없습니다.")


In [ ]:
def print_random_color_recommendation(color_combinations):
  """
  랜덤으로 컬러 조합을 추천하는 함수
  """
  color = random.choice(list(color_combinations.keys()))
  combo = random.choice(color_combinations[color])
  print("\n추천 컬러 조합:")
  print(f"- {color}: {combo}")

In [ ]:
# 전체 옷 목록 가져오기
all_clothes = get_all_clothes()

# 어제 입었던 옷 불러오기 또는 새로 입력받기
previous_clothes = load_previous_clothes()
if previous_clothes:
    print("\n이전에 저장된 어제 입은 옷:")
    print(", ".join(previous_clothes))
    use_previous = input("저장된 어제 옷 정보를 사용하시겠습니까? (y/n): ").strip().lower()
    if use_previous != 'y':
        previous_clothes = []

if not previous_clothes:  # 저장된 옷 정보를 사용하지 않거나 새로 입력하는 경우
    print("\n전체 옷 목록:")
    for idx, cloth in enumerate(all_clothes, start=1):
        print(f"{idx}. {cloth}")

    while True:
        try:
            selected = input("\n어제 입은 옷의 번호를 쉼표로 구분하여 입력하세요 (예: 1,3,5): ")
            selected_indices = [int(x.strip()) for x in selected.split(",") if x.strip().isdigit()]
            if all(1 <= idx <= len(all_clothes) for idx in selected_indices):
                previous_clothes = [all_clothes[idx - 1] for idx in selected_indices]
                break
            else:
                print("목록에 없는 번호가 포함되어 있습니다. 다시 입력해주세요.")
        except ValueError:
            print("유효한 숫자를 입력하세요.")

    save_previous_clothes(previous_clothes)  # 새로운 어제 옷 저장

print(f"\n어제 입은 옷: {', '.join(previous_clothes)}")

# 한국어 도시 이름과 영어 도시 이름 매핑
city_mapping = {
    "서울": "Seoul",
    "부산": "Busan",
    "대구": "Daegu",
    "인천": "Incheon",
    "광주": "Gwangju",
    "대전": "Daejeon",
    "울산": "Ulsan",
    "세종": "Sejong",
    "경기도": "Gyeonggi-do",
    "강원도": "Gangwon-do",
    "충청북도": "Chungcheongbuk-do",
    "충청남도": "Chungcheongnam-do",
    "전라북도": "Jeollabuk-do",
    "전라남도": "Jeollanam-do",
    "경상북도": "Gyeongsangbuk-do",
    "경상남도": "Gyeongsangnam-do",
    "제주도": "Jeju-do"
}

# 사용자 입력
city_kor = input("\n날씨를 확인할 대한민국의 시/도를 입력하세요: ").strip()

# 한국어를 영어로 변환
city = city_mapping.get(city_kor, None)
if not city:
    print("잘못된 도시 이름입니다. 다시 입력해주세요.")
else:
    pass

def fetch_weather_and_process():
    sky, temp = forecast(city)  # api를 통해 날씨 데이터를 가져오기
    print(f"\n{city}의 현재 날씨: {sky}, 온도: {convert_temperature(temp)}")

    # 사용자 입력으로 계절 입력
    season = input("계절을 입력하세요 (봄, 여름, 가을, 겨울): ").strip().lower()

    # 가을 또는 겨울을 입력한 경우 "가을,겨울"로 인식
    if season in ["가을", "겨울"]:
        season = "가을,겨울"
    elif season not in SEASONAL_COLOR_COMBINATIONS:
        print("잘못된 계절 입력입니다.")
        return

    # 날씨에 따른 옷 추천 목록 생성
    recommendations = get_clothing_by_weather(temp)

    # 계절에 따른 컬러 조합 추천
    color_recommendations = SEASONAL_COLOR_COMBINATIONS[season]

    # 결과 출력
    print_random_recommendation(recommendations, previous_clothes)
    print_random_color_recommendation(color_recommendations)

# 실행
fetch_weather_and_process()


이전에 저장된 어제 입은 옷:
숏패딩/숏헤비 아우터, 롱패딩/롱헤비 아우터
저장된 어제 옷 정보를 사용하시겠습니까? (y/n): y

어제 입은 옷: 숏패딩/숏헤비 아우터, 롱패딩/롱헤비 아우터

날씨를 확인할 대한민국의 시/도를 입력하세요: 서울

Seoul의 현재 날씨: 맑음, 온도: 4(C)
계절을 입력하세요 (봄, 여름, 가을, 겨울): 겨울

오늘의 추천 옷:
- Outer: [002024] 겨울 더블 코트 - https://www.musinsa.com/category/002024
- Top: [001006] 니트/스웨터 - https://www.musinsa.com/category/001006
- Bottom: [003002] 데님 팬츠 - https://www.musinsa.com/category/003002

추천 컬러 조합:
- 베이지 베이스: 베이지 + 브라운
